In [1]:
import pyspark
import os
from datetime import datetime
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

In [3]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.13.0 pyspark-shell'
sc = SparkSession.builder.appName("Lab2").master("yarn").getOrCreate()

In [5]:
!hadoop fs -put * . # перемещаем файлы

put: `lab1.ipynb': File exists


In [7]:
prog_langes = sc.read.csv("programming-languages.csv") # чтение файла методом read.csv

# Создадим список из имен языков программирования
prog_langes_list = [str(x[0]) for x in prog_langes.collect()]
prog_langes_list[:10] # первые 10 языков

['name',
 'A# .NET',
 'A# (Axiom)',
 'A-0 System',
 'A+',
 'A++',
 'ABAP',
 'ABC',
 'ABC ALGOL',
 'ABSET']

In [10]:
# читаем xml файл используя метод read с указанием формата - format("xml), построчно - options(rowTag="row"), указание файла - load("path").
posts_sample = sc.read.format("xml").options(rowTag="row").load("posts_sample.xml")
posts_sample.first()

Row(_AcceptedAnswerId=7, _AnswerCount=13, _Body="<p>I want to use a track-bar to change a form's opacity.</p>\n\n<p>This is my code:</p>\n\n<pre><code>decimal trans = trackBar1.Value / 5000;\nthis.Opacity = trans;\n</code></pre>\n\n<p>When I build the application, it gives the following error:</p>\n\n<blockquote>\n  <p>Cannot implicitly convert type <code>'decimal'</code> to <code>'double'</code></p>\n</blockquote>\n\n<p>I tried using <code>trans</code> and <code>double</code> but then the control doesn't work. This code worked fine in a past VB.NET project.</p>\n", _ClosedDate=None, _CommentCount=2, _CommunityOwnedDate=datetime.datetime(2012, 10, 31, 20, 42, 47, 213000), _CreationDate=datetime.datetime(2008, 8, 1, 2, 42, 52, 667000), _FavoriteCount=48, _Id=4, _LastActivityDate=datetime.datetime(2019, 7, 19, 5, 39, 54, 173000), _LastEditDate=datetime.datetime(2019, 7, 19, 5, 39, 54, 173000), _LastEditorDisplayName='Rich B', _LastEditorUserId=3641067, _OwnerDisplayName=None, _OwnerUserI

In [11]:
# Функция language_search переводит весь текст в нижний регистр и ищет название языка программирования в каждой строке
# если язык был найден, то создается кортеж, иначе None.
def language_search(x):
  tag = None
  for language in prog_langes_list:
    if "<" + language.lower() + ">" in x._Tags.lower():
      tag = language
      break
  if tag is None:
    return None
  return (x._Id, tag)

# фильтрация даты необходимого диапазона с 2010 по 2020
def date_filter(x, year):
  start = datetime(year=year, month=1, day=1)
  end = datetime(year=year, month=12, day=31)
  CreationDate = x._CreationDate
  return CreationDate >= start and CreationDate <= end

In [16]:
# Убираем пустые значения и оставляет нужный диапазон, находим язык программирования в каждой строке, иначе убираем пустые значения
# смотрим сколько раз упоминался каждый язык программирования в каждом году, сортируем по количеству повторений 
# сортируем от большего к меньшему по количеству упоминаний
#  помощью метода toDF и с использованием col форматируем полученные результаты.
result = {}
for year in range(2010, 2020):
  result[year] = posts_sample.rdd\
      .filter(lambda x: x._Tags is not None and date_filter(x, year)).map(language_search)\
      .filter(lambda x: x is not None).keyBy(lambda x: x[1])\
      .aggregateByKey(0, lambda x, y: x + 1, lambda x1, x2: x1 + x2,)\
      .sortBy(lambda x: x[1], ascending=False).toDF()
  result[year] = result[year].select(col("_1").alias("Programming_language"),col("_2").alias(f"Mentions_in_{year}")).limit(10)
  result[year].show()

+--------------------+----------------+
|Programming_language|Mentions_in_2010|
+--------------------+----------------+
|                Java|              52|
|          JavaScript|              44|
|                 PHP|              42|
|              Python|              25|
|         Objective-C|              22|
|                   C|              20|
|                Ruby|              11|
|              Delphi|               7|
|         AppleScript|               3|
|                   R|               3|
+--------------------+----------------+

+--------------------+----------------+
|Programming_language|Mentions_in_2011|
+--------------------+----------------+
|                 PHP|              97|
|                Java|              92|
|          JavaScript|              82|
|              Python|              35|
|         Objective-C|              33|
|                   C|              24|
|                Ruby|              17|
|                Perl|               8|

In [19]:
# используем метод write.save() для сохранения данных, также указывем нужный формат с помощью format()
for year in result.keys():
    result[year].write.format("parquet").save(f"Top_{year}")

In [20]:
sc.stop()